<a href="https://colab.research.google.com/github/larissafcerqueira/soulcode/blob/main/Brasil_IncendiosFlorestais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Terra Brasilis - Queimadas

Fonte : https://terrabrasilis.dpi.inpe.br/queimadas/portal/dados-abertos/

# Sumário

1 - Ingestão de Dados
  - Extração de Dados
  - Transformação de Dados
  - Carregamento de Dados

2 - Mais arquivos (replicando em funções)

# 1 - Ingestão de dados

## Extração de Dados

In [ ]:
import pandas as pd

# Aqui, solicitamos ao pandas que leia o arquivo com nome "focos_mensal_br_202308.csv"
df = pd.read_csv("input/focos_mensal_br_202308.csv")

# depois, armazenamos esse dataframe na variável "df" (df significa dataframe)

In [ ]:
# Com df.head() ou df.tail(),
# Podemos visualizar uma parcela pequena dos dados carregados
df.head(2)

,id,lat,lon,data_hora_gmt,satelite,municipio,estado,pais,municipio_id,estado_id,pais_id,numero_dias_sem_chuva,precipitacao,risco_fogo,bioma,frp
0,d3fd9911-d828-3228-9914-da431c1df183,-3.7637,-48.0287,2023-08-01 00:05:28,GOES-16,PARAGOMINAS,PARÁ,Brasil,1505502,15,33,43,0.0,1.00,Amazônia,96.6
1,0347d6da-556a-3a3f-9e5a-c3844a8587a3,-5.8654,-45.5057,2023-08-01 00:05:39,GOES-16,GRAJAÚ,MARANHÃO,Brasil,2104800,21,33,75,0.0,0.98,Cerrado,587.0


In [ ]:
# Com df.info(), podemos visualizar os tipos de dados
# Isso será necessário para a etapa de Transformação de dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709556 entries, 0 to 709555
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     709556 non-null  object 
 1   lat                    709556 non-null  float64
 2   lon                    709556 non-null  float64
 3   data_hora_gmt          709556 non-null  object 
 4   satelite               709556 non-null  object 
 5   municipio              709556 non-null  object 
 6   estado                 709556 non-null  object 
 7   pais                   709556 non-null  object 
 8   municipio_id           709556 non-null  int64  
 9   estado_id              709556 non-null  int64  
 10  pais_id                709556 non-null  int64  
 11  numero_dias_sem_chuva  709556 non-null  int64  
 12  precipitacao           709556 non-null  float64
 13  risco_fogo             709556 non-null  float64
 14  bioma                  709555 non-nu

## Transformação de Dados

Podemos visualizar algumas variáveis que contém tipos de dados estranhos, que precisam ser tratados para serem utilizados corretamente.

Alguns deles são:
data_hora_gmt (com tipo object) -> deveria ser uma data

satelite, municipio, estado, pais (com tipo object) -> todos esses deveriam ser tipos string

Confira a documentação dos dtypes do pandas para mais informações:

https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#dtypes

In [ ]:
# Mudando o dtype das colunas object para string
df['id'] = df['id'].astype('string')
df['satelite'] = df['satelite'].astype('string')
df['municipio'] = df['municipio'].astype('string')
df['estado'] = df['estado'].astype('string')
df['pais'] = df['pais'].astype('string')
df['bioma'] = df['bioma'].astype('string')

In [ ]:
# Mudando o dtype das colunas object para datetype
df['data_hora_gmt'] = pd.to_datetime(df['data_hora_gmt'])

# Vamos construir colunas de ano, mes e dia para facilitar o agrupamento posterior
df['ano'] = df['data_hora_gmt'].dt.year
df['mes'] = df['data_hora_gmt'].dt.month
df['dia'] = df['data_hora_gmt'].dt.day

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709556 entries, 0 to 709555
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   id                     709556 non-null  string        
 1   lat                    709556 non-null  float64       
 2   lon                    709556 non-null  float64       
 3   data_hora_gmt          709556 non-null  datetime64[ns]
 4   satelite               709556 non-null  string        
 5   municipio              709556 non-null  string        
 6   estado                 709556 non-null  string        
 7   pais                   709556 non-null  string        
 8   municipio_id           709556 non-null  int64         
 9   estado_id              709556 non-null  int64         
 10  pais_id                709556 non-null  int64         
 11  numero_dias_sem_chuva  709556 non-null  int64         
 12  precipitacao           709556 non-null  floa

In [ ]:
df.head(2)

,id,lat,lon,data_hora_gmt,satelite,municipio,estado,pais,municipio_id,estado_id,pais_id,numero_dias_sem_chuva,precipitacao,risco_fogo,bioma,frp,ano,mes,dia
0,d3fd9911-d828-3228-9914-da431c1df183,-3.7637,-48.0287,2023-08-01 00:05:28,GOES-16,PARAGOMINAS,PARÁ,Brasil,1505502,15,33,43,0.0,1.00,Amazônia,96.6,2023,8,1
1,0347d6da-556a-3a3f-9e5a-c3844a8587a3,-5.8654,-45.5057,2023-08-01 00:05:39,GOES-16,GRAJAÚ,MARANHÃO,Brasil,2104800,21,33,75,0.0,0.98,Cerrado,587.0,2023,8,1


FRP significa Potência Radiativa de Fogo. É uma medida da intensidade de um incêndio, obtida por satélites que detectam o calor emitido pelas chamas. Quanto maior o FRP, mais intenso o fogo. Essa informação é crucial para monitorar e combater incêndios florestais.

Em resumo: o FRP é como um termômetro para incêndios, indicando a sua "febre".

# Carregamento de Dados SQLAchemy

In [ ]:
import sqlalchemy

# criando engine do sqlalchemy
engine = sqlalchemy.create_engine("sqlite:///meubanco.db")

In [ ]:
query_create_table = """
CREATE TABLE IF NOT EXISTS tabela_dados (
    id VARCHAR(255),
    lat FLOAT,
    lon FLOAT,
    data_hora_gmt TIMESTAMP,
    satelite VARCHAR(255),
    municipio VARCHAR(255),
    estado VARCHAR(255),
    pais VARCHAR(255),
    municipio_id INT,
    estado_id INT,
    pais_id INT,
    numero_dias_sem_chuva INT,
    precipitacao FLOAT,
    risco_fogo FLOAT,
    bioma VARCHAR(255),
    frp FLOAT,
    ano INT,
    mes INT,
    dia INT
);
"""

In [ ]:
try:
  # executar a query1 (CREATE TABLE)
  engine.raw_connection().cursor().execute(query_create_table)
except:
  print("Tabela já existe")

In [ ]:
engine.raw_connection().cursor().execute(query_create_table)

## INSERT

In [ ]:
# INSERT em 'tabela_dados'
# ( colunas )
# VALUES (?)
insert_query = """
INSERT INTO tabela_dados
  (
    id,
    lat,
    lon,
    data_hora_gmt,
    satelite,
    municipio,
    estado,
    pais,
    municipio_id,
    estado_id,
    pais_id,
    numero_dias_sem_chuva,
    precipitacao,
    risco_fogo,
    bioma,
    frp,
    ano,
    mes,
    dia
  )
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

# df = origem = dados extraídos
# df para sql, na tabela de nome 'tabela_dados'
r = df.head(50).to_sql('tabela_dados',
          engine,
          if_exists='append', # se tabela já existir, realizar append
          index=False, # quer salvar o índice do dataframe pandas como coluna?
          method='multi', # 'multi' : passe multiplos valores em uma unica clausula INSERT
          )


if r:
  print("Dados inseridos com sucesso")
  print(f"{r} linhas foram adicionadas")
else:
  print("Erro ao inserir dados")

Dados inseridos com sucesso
50 linhas foram adicionadas


# SQLAlchemy (delete & dispose)

In [ ]:
delete_query = "DROP TABLE tabela_dados"

# deleta a tabela
engine.raw_connection().cursor().execute(delete_query)

# Mais arquivos (replicando em funções)

In [ ]:
import os
import pandas as pd

def processo_etl(nome_arquivo:str):
############################## INGESTÃO DE DADOS ############

  # criar pasta input
  try:
    os.makedirs('input')
  except FileExistsError:
    pass

  # inicia ingestão

  df = pd.read_csv("input/"+nome_arquivo)

############################## TRANSFORMAÇÃO DE DADOS ############

  df['id'] = df['id'].astype('string')
  df['satelite'] = df['satelite'].astype('string')
  df['municipio'] = df['municipio'].astype('string')
  df['estado'] = df['estado'].astype('string')
  df['pais'] = df['pais'].astype('string')
  df['bioma'] = df['bioma'].astype('string')

  df['data_hora_gmt'] = pd.to_datetime(df['data_hora_gmt'])

  df['ano'] = df['data_hora_gmt'].dt.year
  df['mes'] = df['data_hora_gmt'].dt.month
  df['dia'] = df['data_hora_gmt'].dt.day

############################## CARREGAMENTO DE DADOS SQLALCHEMY ############
  # Ingestão no SQL Alchemy

  # criando engine do sqlalchemy
  engine = sqlalchemy.create_engine("sqlite:///meubanco.db")


  try:
    # executar a query1 (CREATE TABLE)
    engine.raw_connection().cursor().execute(query_create_table)
  except:
    print("Tabela já existe")

  # df = origem = dados extraídos
  # df para sql, na tabela de nome 'tabela_dados'
  r = df.head(50).to_sql('tabela_dados',
          engine,
          if_exists='append', # se tabela já existir, realizar append
          index=False, # quer salvar o índice do dataframe pandas como coluna?
          method='multi', # 'multi' : passe multiplos valores em uma unica clausula INSERT
          )

  # deletar nome_arquivo da pasta input
  # os.remove(f'input/{nome_arquivo}')

  if r:
    print("Dados inseridos com sucesso")
    print(f"{r} linhas foram adicionadas")
  else:
    print("Erro ao inserir dados")

  return "finalizado com sucesso"

In [ ]:
print("Iniciando processo de ETL")

for arquivo in os.listdir('input'):
  print(f"arquivo encontrado com nome {arquivo}")

  print("processar arquivo...")
  r = processo_etl(
      nome_arquivo = arquivo
      )

  print("resultados do processamento:")

  print(r)

Iniciando processo de ETL
arquivo encontrado com nome focos_mensal_br_202408.csv
processar arquivo...
Dados inseridos com sucesso
50 linhas foram adicionadas
resultados do processamento:
finalizado com sucesso
arquivo encontrado com nome focos_mensal_br_202308.csv
processar arquivo...
Dados inseridos com sucesso
50 linhas foram adicionadas
resultados do processamento:
finalizado com sucesso


# SQLAlchemy (delete & dispose)

In [ ]:
delete_query = "DROP TABLE tabela_dados"

# deleta a tabela
engine.raw_connection().cursor().execute(delete_query)

In [ ]:
engine.dispose()